In [154]:
import pandas as pd
from IPython.display import display
import numpy as np
import os
import matplotlib.pyplot as plt

In [155]:
# Ptcl bitrate compression table 

# seq_name = "160224_ultimatum2_with_ground"
# seq_name = "160226_mafia1_with_ground"
# seq_name = "160422_haggling1_with_ground"
# seq_name = "160906_ian3_with_ground"

seq_name = "160906_band2_with_ground"
logID = 2

# seq_name = "170307_dance5_with_ground"
# logID = 2

# seq_name = "170915_office1_with_ground"
# logID = 1

# seq_name = "160906_pizza1_with_ground"
# logID = 2

# seq_name = "170915_toddler4_with_ground"
# logID = 2

# Parallelism
# seq_name = "160906_band2_with_ground"
# seq_name = "170307_dance5_with_ground"
# seq_name = "160422_haggling1_with_ground"
# seq_name = "160906_ian3_with_ground"


TARGET_FPS = 15

compression_type = "draco"
# latency_upper_bound = ""            # Default "35ms" for 30 fps 
# latency_upper_bound = "136ms"      # "136ms" 4x parallelism at 30 fps
latency_upper_bound = "70ms"       # "70ms" for 15 fps
comp_table_path = "/datassd/pipeline_ptcl/ptcl_compression_table/"
comp_file_path = os.path.join(comp_table_path, seq_name, "user_log_updated", f"{compression_type}_log{logID}_{latency_upper_bound}.csv")

In [156]:
# Mahimahi trace

bw_type = "server"

# trace_type = "tracep1-scaled10.0"
trace_type = "wifi-25-scaled15.0"

# trace_type = "mh-120-240"
# trace_type = "trace-20.0-scaled10.0"
# trace_type = "mh-120-240-new-scaled3.0"

mmm_bitrate_path = "/datassd/pipeline/mm_bitrate_estimate/"
mm_bitrate_file_path = os.path.join(mmm_bitrate_path, trace_type, seq_name, "user_log_updated", f"bw_{bw_type}_estimate_log{logID}.csv")
# mm_bitrate_file_path = os.path.join(mmm_bitrate_path, trace_type, seq_name, "user_log_updated", f"bw_{bw_type}_estimate.csv")

In [157]:
# Load compression table
comp_table_df = pd.read_csv(comp_file_path)
comp_headers = comp_table_df.columns

# Load mahimahi trace
bitrate_df = pd.read_csv(mm_bitrate_file_path)

In [158]:
comp_table_df

,FrameID,CL,QP,Compression Time (in ms),Decompression Time (in ms),Num Points,Raw Size (in bytes),Compressed Size (in bytes)
0,2,0,0,37,13,251379,3770685,3387089
1,2,0,1,46,20,251379,3770685,379697
2,2,0,3,43,21,251379,3770685,393798
3,2,0,6,48,22,251379,3770685,465482
4,2,0,7,45,22,251379,3770685,511439
...,...,...,...,...,...,...,...,...
116078,5970,7,1,118,27,254148,3812220,62187
116079,5970,8,0,53,14,254148,3812220,3416734
116080,5970,8,1,118,27,254148,3812220,62187
116081,5970,9,0,53,15,254148,3812220,3416734


In [159]:
bitrate_df

,FrameID,BWE(in kbps),CBWE(in kbps),DBWE(in kbps)
0,2,91980,38039,47199
1,3,91620,39155,45907
2,4,91080,13012,78068
3,5,90720,12960,77760
4,6,90720,12960,77760
...,...,...,...,...
5964,5966,88560,26303,54475
5965,5967,88380,26576,54079
5966,5968,88200,26495,53992
5967,5969,87660,25871,54066


In [160]:
def best_quality(comp_frame_df, target_frame_size, target_latency):
    df = comp_frame_df.copy()
    df = df[df["Compressed Size (in bytes)"] <= target_frame_size]
    df = df[df["Compression Time (in ms)"] <= target_latency]
    df = df[df["Decompression Time (in ms)"] <= target_latency]

    if df.empty:
        return None

    # Best quality is highest QP in draco
    maxQP = df["QP"].max()

    # Extract maxQP rows
    maxQP_df = df[df["QP"] == maxQP]
    return maxQP_df

# This is incorrect ,since the df has been filtered for compression latency <= 35 ms
def check_bw_violation(comp_frame_df, target_frame_size):
    df = comp_frame_df.copy()
    df = df[df["Compressed Size (in bytes)"] <= target_frame_size]

    if df.empty:
        return True

    return False

"""
If compute is bottleneck, then bw violation is not possible
If compute is not the bottleneck, then bw violation is possible
"""
def check_compute_bw_violation(comp_frame_df, target_latency, target_frame_size):
    compute_violation = False
    bw_violation = False

    df = comp_frame_df.copy()

    df = df[df["Compression Time (in ms)"] <= target_latency]
    df = df[df["Decompression Time (in ms)"] <= target_latency]
    
    if df.empty:
        compute_violation = True
        return compute_violation, bw_violation
    
    df = df[df["Compressed Size (in bytes)"] <= target_frame_size]

    if df.empty:
        bw_violation = True
        return compute_violation, bw_violation
    
    return compute_violation, bw_violation

In [161]:
codec_parallelism = 1.0
target_latency = 1000.0 * codec_parallelism / float(TARGET_FPS)    # in ms
stalls = 0.0
compute_violations = 0.0
bw_violations = 0.0
num_frames = 0

# Write ABR bitrate selection

abr_folder_path = os.path.join(comp_table_path, seq_name, "user_log_updated", trace_type)
if not os.path.exists(abr_folder_path):
    os.makedirs(abr_folder_path)

abr_file_path = os.path.join(abr_folder_path, f"{compression_type}_parallel{codec_parallelism}_abr_log{logID}_lat{latency_upper_bound}_fps{TARGET_FPS}.csv")
abr_file = open(abr_file_path, "w")

# Write to ABR table
for names in comp_headers:
    abr_file.write(names + ",")
abr_file.write("Target Size (in bytes)\n")
abr_file.flush()

for frameID in bitrate_df["FrameID"]:
    bwe, cbwe, dbwe = bitrate_df.loc[bitrate_df["FrameID"] == frameID, ["BWE(in kbps)", "CBWE(in kbps)", "DBWE(in kbps)"]].values[0]
    comp_frame_df = comp_table_df.loc[comp_table_df["FrameID"] == frameID]
    if comp_frame_df.empty:                                                     # No compression data for this frame
        print(f"No compression data for frame {frameID}. Skipping.")
        continue
    
    num_frames += 1.0

    target_frame_size_bytes = bwe * 1000.0 / (8.0 * float(TARGET_FPS))                 # bwe is in kbps, so we need to convert to bytes
    print(f"FrameID: {frameID}, BWE: {bwe}, CBWE: {cbwe}, DBWE: {dbwe}, Target frame size: {target_frame_size_bytes} bytes")

    cv, bwv = check_compute_bw_violation(comp_frame_df, target_latency, target_frame_size_bytes)
    if cv:
        compute_violations += 1.0
    if bwv:
        bw_violations += 1.0

    maxQP_df = best_quality(comp_frame_df, target_frame_size_bytes, target_latency)
    if(maxQP_df is None):
        # print("No valid compression found for this frame. Skipping.")
        abr_file.write(f"{frameID},-1,-1,-1,-1,0,0,0,{target_frame_size_bytes}\n")
        stalls += 1.0
        continue
    
    # display(maxQP_df)
    assert(maxQP_df["FrameID"].values[0] == frameID)

    cl = maxQP_df["CL"].values[0]
    qp = maxQP_df["QP"].values[0]
    comp_time = maxQP_df["Compression Time (in ms)"].values[0]
    decomp_time = maxQP_df["Decompression Time (in ms)"].values[0]
    num_points = maxQP_df["Num Points"].values[0]
    raw_size = maxQP_df["Raw Size (in bytes)"].values[0]
    comp_size = maxQP_df["Compressed Size (in bytes)"].values[0]
    target_size = target_frame_size_bytes

    # Write to ABR file
    abr_file.write(f"{frameID},{cl},{qp},{comp_time},{decomp_time},{num_points},{raw_size},{comp_size},{target_size}\n")

abr_file.flush()
abr_file.close()

stalls_percent = stalls / num_frames * 100.0
compute_violations_percent = compute_violations / num_frames * 100.0
bw_violations_percent = bw_violations / num_frames * 100.0

print(f"Total frames: {num_frames}, Stalls: {stalls}, Compute Violations: {compute_violations}, BW Violations: {bw_violations}")
print(f"Stalls: {stalls_percent}%, Compute Violations: {compute_violations_percent}%, BW Violations: {bw_violations_percent}%")

FrameID: 2, BWE: 91980, CBWE: 38039, DBWE: 47199, Target frame size: 766500.0 bytes
FrameID: 3, BWE: 91620, CBWE: 39155, DBWE: 45907, Target frame size: 763500.0 bytes
FrameID: 4, BWE: 91080, CBWE: 13012, DBWE: 78068, Target frame size: 759000.0 bytes
FrameID: 5, BWE: 90720, CBWE: 12960, DBWE: 77760, Target frame size: 756000.0 bytes
FrameID: 6, BWE: 90720, CBWE: 12960, DBWE: 77760, Target frame size: 756000.0 bytes
FrameID: 7, BWE: 90540, CBWE: 12935, DBWE: 77605, Target frame size: 754500.0 bytes
FrameID: 8, BWE: 90180, CBWE: 12883, DBWE: 77297, Target frame size: 751500.0 bytes
FrameID: 9, BWE: 89640, CBWE: 12806, DBWE: 76834, Target frame size: 747000.0 bytes
FrameID: 10, BWE: 89460, CBWE: 12780, DBWE: 76680, Target frame size: 745500.0 bytes
FrameID: 11, BWE: 89460, CBWE: 12780, DBWE: 76680, Target frame size: 745500.0 bytes
FrameID: 12, BWE: 89280, CBWE: 12755, DBWE: 76525, Target frame size: 744000.0 bytes
FrameID: 13, BWE: 89100, CBWE: 12400, DBWE: 67113, Target frame size: 742